# Configuration Management Script Explanation

This notebook explains the script that manages configurations for web scraping and data import processes. It uses `configparser` to read, write, and validate configuration settings.

## 1. Introduction

The script is designed to:
1. Read configuration values from an INI file.
2. Set default configuration values.
3. Validate the configuration file and ensure it has valid values.
4. Provide decorators to conditionally execute functions based on configuration settings.

## 2. Import Necessary Libraries

The script imports the `configparser` library to handle INI files.


import configparser


## 3. Define `read_config` Function

The `read_config` function reads the configuration file and returns a dictionary with the configuration values.

### Explanation of the Code

- Creates a `ConfigParser` object.
- Reads the configuration file `config.ini`.
- Retrieves and returns the configuration values as a dictionary.


In [ ]:
def read_config():
    # Create a ConfigParser object
    config = configparser.ConfigParser()

    # Read the configuration file
    config.read("config.ini")

    # Return a dictionary with the retrieved values
    config_values = {
        "get_data": config.getboolean("Webscraping", "get_data"),
        "extract_data": config.getboolean("Webscraping", "extract_data"),
        "users": config.getboolean("DataImport", "users"),
        "groups": config.getboolean("DataImport", "groups"),
        "memberships": config.getboolean("DataImport", "memberships"),
    }

    return config_values


## 4. Define `default_config` Function

The `default_config` function sets default configuration values and writes them to the `config.ini` file.

### Explanation of the Code

- Creates a `ConfigParser` object.
- Sets default values for the `Webscraping` and `DataImport` sections.
- Writes the default configuration values to `config.ini`.


In [ ]:
def default_config():
    config = configparser.ConfigParser()

    config["Webscraping"] = {"get_data": "True", "extract_data": "True"}
    config["DataImport"] = {
        "users": "True",
        "groups": "True",
        "memberships": "True",
    }

    # Write the configuration to a file
    with open("config.ini", "w") as configfile:
        config.write(configfile)


## 5. Define `validate_config` Function

The `validate_config` function validates the configuration file and ensures it has valid values. If the configuration file is missing or contains invalid values, it rewrites the file with default values.

### Explanation of the Code

- Defines default values for the configuration.
- Reads the configuration file `config.ini`.
- Checks for missing sections or keys and validates the values.
- If any validation fails, rewrites the configuration file with default values.
- Returns the configuration values as a dictionary.


In [ ]:
def validate_config():
    default_values = {
        "Webscraping": {"get_data": True, "extract_data": True},
        "DataImport": {
            "users": True,
            "groups": True,
            "memberships": True,
        },
    }

    config = configparser.ConfigParser()
    config.read("config.ini")

    try:
        for section, keys in default_values.items():
            if section not in config:
                raise ValueError(f"Missing section: {section}")
            for key, default_value in keys.items():
                if key not in config[section]:
                    raise ValueError(f"Missing key: {key} in section: {section}")
                if config.get(section, key).lower() not in ["true", "false"]:
                    raise ValueError(
                        f"Invalid value for key: {key} in section: {section}"
                    )

        # If all values are valid, parse them as booleans
        config_values = {
            section: {key: config.getboolean(section, key) for key in keys}
            for section, keys in default_values.items()
        }

    except Exception as e:
        print(f"Exception caught: {e}")
        print("Rewriting config with default values.")
        default_config()
        config_values = default_values

    return config_values


## 6. Define Configuration Decorators

The configuration decorators conditionally execute functions based on configuration settings.

### Explanation of the Code

- `config_check_web`: Decorator that checks if web scraping should be executed.
- `config_check_import`: Decorator that checks if data import should be executed.


In [ ]:
config_web = validate_config()["Webscraping"]
config_data = validate_config()["DataImport"]

def config_check_web(config):
    def decorator(func):
        def wrapper(*args, **kwargs):
            if config:
                print("Executing " + func.__name__ + "...")
                return func(*args, **kwargs)
            else:
                print("Skipping " + func.__name__ + "...")
                return None

        return wrapper

    return decorator

def config_check_import(key):
    def decorator(func):
        async def wrapper(config, data, db_writer, gql_func):
            if config.get(key, False):
                print(f"Executing {key} import...")
                await func(data[key], db_writer, gql_func)
            else:
                print(f"Skipping {key} import...")

        return wrapper

    return decorator


## 7. Conclusion

In this notebook, we have broken down the configuration management script, explaining each part and its purpose. This script is useful for managing configuration settings for web scraping and data import processes.

Ensure you have the necessary configuration file (`config.ini`) in place for this script to run correctly. If the file is missing or contains invalid values, the script will rewrite it with default values.


